In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gurobipy as gp
from gurobipy import GRB
from sklearn.cluster import SpectralClustering

In [2]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 40
DATA_NAME = "harmonic9"
TOPO_NAME = "0"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## 1. Model

In [3]:
m = gp.Model("IEEETII2020")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01


In [4]:
## Unicast

M = int(1e16)
NG = 5
ITRN = 100

Network model

In [5]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['tproc'] = np.ceil(row['t_proc'])
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

Task model

$$s_m^r \equiv\left(s r_m, d_m, p_m, s i_m, \mathscr{P}_m^r\right)$$

In [6]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [7]:
task_attr = {}
task_var = {}

## Assume task is strictly periodic
for i, row in task.iterrows():
    task_var.setdefault(i, {})
    task_attr.setdefault(i, {})
    task_attr[i]['sr'] = row['src']
    task_attr[i]['d'] = row['dst']
    task_attr[i]['p'] = row['period']
    task_attr[i]['si'] = row['size']
    task_attr[i]['dl'] = row['deadline']

## 2. Graph based stream partitioning

In [8]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    paths = []
    for node in set(np.reshape(np.argwhere(graph[start] > 0),  -1)):
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths     

In [9]:
paths = {}
for i in task_attr:
    paths[i] = find_all_paths(net, task_attr[i]['sr'],eval(task_attr[i]['d'])[0])
    for k in range(len(paths[i])):
        paths[i][k] = list({x: int(eval(str(paths[i][k]))[h+1]) for h, x in enumerate(eval(str(paths[i][k]))[:-1])}.items())
        

In [10]:
doc_net = np.zeros(shape = (len(task), len(task)))
for i in task_attr:
    for j in task_attr:
        if i < j:
            doc_net[i][j] = doc_net[j][i] = len(set([x for y in paths[i] for x in y]) & set([x for y in paths[j] for x in y])) * \
            task_attr[i]['si'] * task_attr[j]['si'] / task_attr[i]['p'] * task_attr[j]['p'] 

In [11]:
cluster = SpectralClustering(n_clusters = NG)

In [12]:
task_group = cluster.fit_predict(doc_net)

/home/chuanyu/.local/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:658: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
/home/chuanyu/.local/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [421]:
# task_group = [x for y in [[i] * int(np.ceil(len(task) / NG)) for i in range(NG)] for x in y]
# task_group = [0] * len(task)

## 3. DAMR Procedures

In [422]:
opt = [0 for i in task_attr]
costs = [sum([ 
    len(set(paths[i][opt[i]]) & set(paths[j][opt[j]])) * \
            task_attr[i]['si'] * task_attr[j]['si'] / task_attr[i]['p'] * task_attr[j]['p'] 
     for j in task_attr if i != j]) for i in task_attr]

for it in range(ITRN):
    i = np.argmax(costs)
    best = costs[i]
    m_star = opt[i]
    for m in range(len(paths[i])):
        if m != opt[i]:
            cost = sum([len(set(paths[i][m]) & set(paths[j][opt[j]])) * \
            task_attr[i]['si'] * task_attr[j]['si'] / task_attr[i]['p'] * task_attr[j]['p'] for j in task_attr])
            if cost < best:
                best = cost
                m_star = m
    opt[i] = m_star

In [423]:
for i in task_attr:
    task_attr[i]['r'] = paths[i][opt[i]]

# for i in task_attr:
#     path = eval(str(next(bfs_paths(net, int(row['src']), eval(row['dst'])[0]))))
#     task_attr[i]['r'] = [(x, path[i + 1]) for i,x in enumerate(path[:-1])]
    

In [424]:
## Assume task is strictly periodic
for i, row in task.iterrows():
    task_var.setdefault(i, {})
    route = task_attr[i]['r']
    for _i, link in enumerate(route):
        task_var[i].setdefault(link, {})
        task_var[i][link]['dtrans'] = row['size'] * 8
        if _i == 0:## This one must not cantains processing delay
            
            task_var[i][link]['D'] = task_var[i][link]['dtrans']
        else:
            task_var[i][link]['D'] = task_var[i][route[_i - 1]]['D'] + net_var[link[0]]['tproc'] + task_var[i][link]['dtrans']

## 4. Constraints

\begin{aligned}
\min C_{m a x} & \\
\text { subject to } & \\
\forall\left\{O_{i, k}, O_{j, l}\right\} \in K: & \\
t_{j}-t_{i}-D_{i, k}+D_{j, l-1}+d^{\text {prop }}+d^{\text {proc }} & \leq c x_{i, k, j, l} \\
\forall\left\{O_{i, k}, O_{j, l}\right\} \in K: & \\
t_{i}-t_{j}-D_{j, l}+D_{i, k-1}+d^{p r o p}+d^{\text {proc }} & \leq c\left(1-x_{i, k, j, l}\right)
\end{aligned}

In [425]:
solutions = [None for i in task_var]

for epoch in range(NG):
    m = gp.Model("IEEEII2022_%d"%epoch)
    m.Params.LogToConsole = 0
    t = m.addMVar(shape=(len(task)), vtype=GRB.INTEGER , name="release")
    for i in [i for i in task_var if task_group[i] == epoch]:
        end_link = task_attr[i]['r'][-1]
        m.addConstr(0 <= t[i])
        m.addConstr(t[i] <= task.loc[i]['period'] - task_var[i][end_link]['D'])
    ## Add constraint within task subgroup
    for i, j in [(i,j) for i in task_var for j in task_var if task_group[i] == epoch]:
        ir, jr = task_attr[i]['r'], task_attr[j]['r']
        lcm = np.lcm(task.loc[i].period, task.loc[j].period)
        for k, l in [(k, l) for k in range(len(ir)) for l in range(len(jr))]:
            if i != j and ir[k] == jr[l] and task_group[j] == epoch:
                for a, b in [(a,b) for a in range(0, int(lcm / task.loc[i].period)) for b in range(0, int(lcm / task.loc[j].period))]:
                    temp = m.addVar(vtype=GRB.BINARY, name = "%d%d%d%d"%(i,j,k,l))
                    m.addConstr(
                        (t[j] + b * task.loc[j].period) - (t[i] + a * task.loc[i].period) - task_var[i][ir[k]]['D'] + task_var[i][ir[k]]['dtrans'] 
                        + task_var[j][jr[l]]['D'] <= M * temp
                    )
                    m.addConstr(
                        (t[i] + a * task.loc[i].period) - (t[j] + b * task.loc[j].period) - task_var[j][jr[l]]['D'] + task_var[j][jr[l]]['dtrans'] 
                        + task_var[i][ir[k]]['D'] <= M * (1 - temp)
                    )
            if i != j and ir[k] == jr[l] and task_group[j] < epoch:
                for a, b in [(a,b) for a in range(0, int(lcm / task.loc[i].period)) for b in range(0, int(lcm / task.loc[j].period))]:
                    temp = m.addVar(vtype=GRB.BINARY, name = "%d%d%d%d"%(i,j,k,l))
                    m.addConstr(
                        (solutions[j] + b * task.loc[j].period) - (t[i] + a * task.loc[i].period) - task_var[i][ir[k]]['D'] + task_var[i][ir[k]]['dtrans'] 
                        + task_var[j][jr[l]]['D'] <= M * temp
                    )
                    m.addConstr(
                        (t[i] + a * task.loc[i].period) - (solutions[j] + b * task.loc[j].period) - task_var[j][jr[l]]['D'] + task_var[j][jr[l]]['dtrans'] 
                        + task_var[i][ir[k]]['D'] <= M * (1 - temp)
                    )
    m.optimize()
    for i in [i for i in task_var if task_group[i] == epoch]:
        solutions[i] = t[i].x

## Output Schedule

In [ ]:
GCL = []
for i in task_var:
    path = task_attr[i]['r']
    for e in path:
        start = solutions[i] + task_var[i][e]['D'] - task_var[i][e]['dtrans']
        end = start + task_var[i][e]['dtrans']
        queue = 0
        tt = task.loc[i, 'period']
        for k in range(int(LCM / tt)):
            GCL.append(
                [e, queue, int(start + k * tt) * macrotick, int(end + k * tt) * macrotick, LCM * macrotick]
            )

In [ ]:
## Offset
OFFSET = []
for i in task_var:
    offset = solutions[i]
    OFFSET.append(
        [i, 0, (task.loc[i,'period'] - offset) * macrotick]
    )    

In [ ]:
ROUTE = []
for i in task_attr:
    for link in task_attr[i]['r']:
        ROUTE.append([i, link])

In [ ]:
QUEUE = []
for i in task_attr:
    for e in task_attr[i]['r']:
        QUEUE.append([i, 0, e, 0])

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("IEEETII2020-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("IEEETII2020-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("IEEETII2020-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("IEEETII2020-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)